# Giới thiệu về Transformers

## Bài 1: Khôi phục Masked Token (Masked Language Modeling)

**Yêu cầu**: Sử dụng pipeline `fill-mask` để dự đoán từ bị thiếu trong câu sau: *Hanoi is the [MASK] of Vietnam"*

In [ ]:
from transformers import pipeline

# 1. Tải pipeline "fill-mask"
# Pipeline này sẽ tự động tải một mô hình mặc định phù hợp (thường là một biến thể của BERT)
mask_filler = pipeline("fill-mask")

# 2. Câu đầu vào với token [MASK]
input_sentence = "Hanoi is the <mask> of Vietnam."

# 3. Thực hiện dự đoán
# top_k=5 yêu cầu mô hình trả về 5 dự đoán hàng đầu
predictions = mask_filler(input_sentence, top_k=5)

# 4. In kết quả
print(f"Câu gốc: {input_sentence}")
for pred in predictions:
    print(f"Dự đoán: '{pred['token_str']}' với độ tin cậy: {pred['score']:.4f}")
    print(f" -> Câu hoàn chỉnh: {pred['sequence']}")

No model was supplied, defaulted to distilbert/distilroberta-base and revision fb53ab8 (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


PipelineException: No mask_token (<mask>) found on the input

1. Mô hình đã dự đoán ra *capital* với độ tin cậy 93%, rất cao so với các lựa chọn còn lại $\rightarrow$ điều này cho thấy mô hình hiểu ngữ cảnh tốt và đưa ra dự đoán chính xác.

2. Các mô hình Encoder-only như BERT phù hợp cho các tác vụ fill-mask vì:
    - Vì BERT dùng cơ chế Masked Language Modeling khi huấn luyện:
        - BERT được huấn luyện bằng cách che một số token bằng <mask>
        - Sau đó mô hình phải dự đoán lại đúng token bị che dựa trên ngữ cảnh hai chiều.
        $\rightarrow$ Vì vậy, BERT được tối ưu hóa tự nhiên cho tác vụ này.
    - Encoder xử lý ngữ cảnh hai chiều:
        - Encoder của BERT: Nhìn cả bên trái và bên phải của token bị che $\rightarrow$ Hiểu toàn bộ câu nên dự đoán tốt hơn.
    - Encoder tạo embedding có khả năng hiểu ngữ nghĩa sâu:
        - Các lớp self-attention trong Encoder: Gán trọng số khác nhau cho các từ liên quan, xây dựng biểu diễn ngữ nghĩa giàu thông tin.

## Bài 2: Dự đoán từ tiếp theo (Next token prediction)

**Yêu cầu**: Sử dụng pipeline `text-generation` để sinh ra phần tiếp theo cho câu: *The best thing about learning NLP is"*

In [2]:
from transformers import pipeline

# 1. Tải pipeline "text-generation"
# Pipeline này sẽ tự động tải một mô hình phù hợp (thường là GPT-2)
generator = pipeline("text-generation")

# 2. Đoạn văn bản mồi
prompt = "The best thing about learning NLP is"

# 3. Sinh văn bản
# max_length: tổng độ dài của câu mồi và phần được sinh ra
# num_return_sequences: số lượng chuỗi kết quả muốn nhận
generated_texts = generator(prompt, max_length=50, num_return_sequences=1)

# 4. In kết quả
print(f"Câu mồi: '{prompt}'")
for text in generated_texts:
    print("Văn bản được sinh ra:")
    print(text['generated_text'])

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\NGUYEN PHUONG BICH\HOC_TAP\NLP\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\NGUYEN PHUONG BICH\.cache\huggingface\hub\models--openai-community--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more de

Câu mồi: 'The best thing about learning NLP is'
Văn bản được sinh ra:
The best thing about learning NLP is that it's not so hard. It's hard because it requires a lot of motivation and time. I've learned over 100 NLP training courses, and have been able to do 100 of them without any problems.

Training a NLP program for a year and then learning a new one takes several years. I have to be a little patient. I have to learn a lot and be able to do it and be flexible. It's going to take me longer if I'm too much and I'm not able to focus on what I'm doing right after I'm done with training.

I have a few other things I have to learn. I've got to build up my NLP skills and learn from my mistakes and learn from my mistakes. But I also have to learn to be a good parent.

I have a lot of things I want to do in life but I'm not really sure where I want to go. I'm not sure what I want to do. I can't really give you a definitive answer.

NLP is really about self-care. You have to get to know yours

1. Kết quả sinh ra 
- Về mặt ngữ pháp: Văn bản được sinh ra có cấu trúc ngữ pháp tương đối đúng, các câu hoàn chỉnh
- Về tính mạch lạc: Văn bản bắt đầu nói về việc học NLP nhưng sau đó lại chuyển sang "self-care" - không liên quan trực tiếp
- Lặp lại: Có nhiều cụm từ bị lặp như "I have to learn", "learn from mistakes"

$\rightarrow$ Văn bản có tính liên kết ngữ pháp nhưng thiếu tính mạch lạc về nội dung và có xu hướng lan man. Đây là đặc điểm thường thấy ở GPT-2 - một mô hình đã cũ (2019) với 117M-1.5B tham số.

2. Các mô hình Decoder-only như GPT phù hợp cho tác vụ này sinh văn bản vì:
- Mô hình được thiết kế với cơ chế causal (autoregressive) attention
    - Mỗi token chỉ có khả năng nhìn một chiều (undirectional), tức là chỉ xem xét các từ đã xuất hiện trước đó
- Khi có một câu mồi, mô hình dự đoán token tiếp theo theo xác suất, sau đó lặp lại quá trình này để sinh ra văn bản liên tục
- Điều này phù hợp cho tác vụ sinh văn bản vì không cần đầu vào nhãn hay cấu trúc encoder-decoder phức tạp, mà chỉ cần dự đoán từ tiếp theo dựa trên ngữ cảnh trước đó.


## Bài 3: Tính toán Vector biểu diễn của câu (Sentence Representation)

**Yêu cầu**: Viết code để tính toán vector biểu diễn cho câu *"This is a sample sentence."*
bằng phương pháp Mean Pooling.

In [3]:
import torch
from transformers import AutoTokenizer, AutoModel

# 1. Chọn một mô hình BERT
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 2. Câu đầu vào
sentences = ["This is a sample sentence."]

# 3. Tokenize câu
# padding=True: đệm các câu ngắn hơn để có cùng độ dài
# truncation=True: cắt các câu dài hơn
# return_tensors='pt': trả về kết quả dưới dạng PyTorch tensors
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# 4. Đưa qua mô hình để lấy hidden states
# torch.no_grad() để không tính toán gradient, tiết kiệm bộ nhớ
with torch.no_grad():
    outputs = model(**inputs)

# outputs.last_hidden_state chứa vector đầu ra của tất cả các token
last_hidden_state = outputs.last_hidden_state
# shape: (batch_size, sequence_length, hidden_size)

# 5. Thực hiện Mean Pooling
# Để tính trung bình chính xác, chúng ta cần bỏ qua các token đệm (padding tokens)
attention_mask = inputs['attention_mask']
mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)
sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
sentence_embedding = sum_embeddings / sum_mask

# 6. In kết quả
print("Vector biểu diễn của câu:")
print(sentence_embedding)
print("\nKích thước của vector:", sentence_embedding.shape)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Vector biểu diễn của câu:
tensor([[-6.3874e-02, -4.2837e-01, -6.6779e-02, -3.8430e-01, -6.5784e-02,
         -2.1826e-01,  4.7636e-01,  4.8659e-01,  4.0658e-05, -7.4274e-02,
         -7.4741e-02, -4.7635e-01, -1.9773e-01,  2.4824e-01, -1.2162e-01,
          1.6678e-01,  2.1045e-01, -1.4576e-01,  1.2636e-01,  1.8635e-02,
          2.4640e-01,  5.7090e-01, -4.7014e-01,  1.3782e-01,  7.3650e-01,
         -3.3808e-01, -5.0330e-02, -1.6452e-01, -4.3517e-01, -1.2900e-01,
          1.6516e-01,  3.4004e-01, -1.4930e-01,  2.2422e-02, -1.0488e-01,
         -5.1916e-01,  3.2964e-01, -2.2162e-01, -3.4206e-01,  1.1993e-01,
         -7.0148e-01, -2.3126e-01,  1.1224e-01,  1.2550e-01, -2.5191e-01,
         -4.6374e-01, -2.7261e-02, -2.8415e-01, -9.9250e-02, -3.7017e-02,
         -8.9192e-01,  2.5005e-01,  1.5816e-01,  2.2701e-01, -2.8497e-01,
          4.5300e-01,  5.0940e-03, -7.9441e-01, -3.1008e-01, -1.7403e-01,
          4.3029e-01,  1.6816e-01,  1.0590e-01, -4.8987e-01,  3.1856e-01,
          3.

c:\Users\NGUYEN PHUONG BICH\HOC_TAP\NLP\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\NGUYEN PHUONG BICH\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


1. Kích thước (chiều) của vector biểu diễn là : [1, 768]. Trong đó:
- `1` : Batch size (số câu trong batch)
- `768` : Chiều của vector embedding

2. Cần sử dụng `attention_mask` khi thực hiện Mean Pooling vì:
- Khi xử lý batch, các câu có độ dài khác nhau cần được đệm (padding) để có cùng độ dài
- Nếu tính Mean Pooling mà không loại bỏ padding thì sẽ tính trung bình cả những vector không mang ý nghĩa từ token [PAD]
    - Padding token không mang ý nghĩa nhưng vẫn có vector 768 chiều: 
    
        Câu ngắn $\rightarrow$ nhiều padding $\rightarrow$ embedding bị loãng. 
    
        Câu dài $\rightarrow$ ít padding $\rightarrow$ embedding phản ánh câu thật hơn. 

    - Mỗi batch có số lượng padding khác nhau: Điều này làm embedding phụ thuộc vào batch, không phụ thuộc vào câu $\rightarrow$ sai về mặt lý thuyết

$\rightarrow$  `attention_mask` được sử dụng khi pooling để:
- Chỉ cộng vector tại vị trí token thật
- Chia cho số token thật (không chia cho seq_len)

Giúp đảm bảo vector câu chỉ phản ánh nội dung thật của câu.